In [4]:
# default_exp core

# Notoma

> Write articles for any static gen blog, in Notion.

In [5]:
#hide
from nbdev.showdoc import *

Notoma is a small tool that works with your Notion database and allows you to turn your Notion pages into a blog, or a website — with any static gen website engine and hosting platform you want.

Notoma is available as a stand alone CLI app, and a Python library. You can use it locally to prepare and preview your articles and commit them, or as a part of your remote build pipeline.

## Imports and dependencies

In [15]:
#export
from typing import List, Dict, Union
from notion.client import NotionClient
from notion.collection import *
from notion.block import *
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

## Config

Where will we store configuration, including sensitive data like auth tokens, and blog db name?

In [65]:
#exports
class Config:
    """
    Wraps Notoma's settings in an object with easier access.
    Settings are loaded from `.env` file, and from the system environment.
    You can override them by providing kwargs when creating an instance of a config.

    `.env` keys are explicit and long, i.e. `NOTOMA_NOTION_TOKEN_V2`. `kwargs` key responsible for the token is just
    `token_v2`.
    """

    def __init__(self, **kwargs):
        """
        Loads config from a `.env` file or system environment.

        You can provide any kwargs you want and they would override environment config values.
        """
        load_dotenv(find_dotenv())
        self.__config = dict(token_v2 = os.environ.get('NOTOMA_NOTION_TOKEN_V2'),
                               blog_url = os.environ.get('NOTOMA_NOTION_BLOG_URL'))

        for k, v in kwargs.items():
            self.__config[k] = v

    @property
    def token_v2(self):
        return self.__config['token_v2']

    @property
    def blog_url(self):
        return self.__config['blog_url']

    def __getitem__(self, key):
        return self.__config[key]

    def __repr__(self):
        return '\n'.join(f'{k}: {v}' for k, v in self.__config.items())

In [48]:
show_doc(Config)

<h2 id="Config" class="doc_header"><code>class</code> <code>Config</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>Config</code>(**\*\*`kwargs`**)

Wraps Notoma's settings in an object with easier access.
Settings are loaded from `.env` file, and from the system environment. 
You can override them by providing kwargs when creating an instance of a config.

`.env` keys are explicit and long, i.e. `NOTOMA_NOTION_TOKEN_V2`. `kwargs` key responsible for the token is just
`token_v2`.

In [66]:
config = Config()

## Notion Client

Provides a thin wrapper around `notion-py` — an API wrapper library for the reverse-engineered Notion API. Pagify users that client to grab the blog contents from Notion.

The client utilizes authentication token from Notion's web version, `token_v2`, that you can grab from your browers cookies. 

> Notice: This token authorizes any code to do anything that you can do in the browser in Notion on your behalf, so don't share it publicly, and don't store it your code or git.

In [49]:
#exports
def notion_client(token_v2:str) -> NotionClient:
    client = NotionClient(token_v2=config.token_v2)
    return client

In [50]:
show_doc(notion_client)

<h4 id="notion_client" class="doc_header"><code>notion_client</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>notion_client</code>(**`token_v2`**:`str`)



In [53]:
client = notion_client(config.token_v2)

## Blog Database

Build a helper that would take the client, search for the provided DB name, and return it, or error out if notfound.

Build a function that returns an iterator over all the pages in that database.

In [54]:
#exports
def notion_blog_database(client: NotionClient, db_url:str) -> Collection:
    """Returns a Notion database, wraped into a `notion.Collection` for easy access to it's rows."""
    return client.get_collection_view(db_url).collection

In [55]:
show_doc(notion_blog_database)

<h4 id="notion_blog_database" class="doc_header"><code>notion_blog_database</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>notion_blog_database</code>(**`client`**:`NotionClient`, **`db_url`**:`str`)

Returns a Notion database, wraped into a `notion.Collection` for easy access to it's rows.

In [56]:
blog = notion_blog_database(client, config.blog_url)

## Page Structure

Build a class or a function that takes the notion page object (from the iterator) and can return the metadata values, and iterate over it's contents.

`Page` is a record in the Blog database. Here's the format that Pagify expects: 

- Page title will be converted to the .md file name. File name will be formatted with dashes instead of spaces, and the page udpated at date will be uppended in YYYY-MM-DD format.

- Published: an optional boolean field. If present, Pagify will ignore pages where published: false. 

- Description: if the Page text starts with a word Desciption, then the whole first paragraph is considered description, and will be added to the markdown file front matter (metadata). 

- Publish at: a datetime field, if present, will be used as published at front matter key in the md file.



In [59]:
page = blog.get_rows()[0]

In [60]:
page.title

'Notoma First Article'

## Converting Page to Markdown

Build a quick conversion of a Notion page to a .md file

In [61]:
#export
def block2md(block:Block, counter:int = 1) -> str:
    """Transforms a Notion Block into a Markdown string."""

    if isinstance(block, TextBlock):
        return block.title

    elif isinstance(block, HeaderBlock):
        return f"# {block.title}"

    elif isinstance(block, SubheaderBlock):
        return f"## {block.title}"

    elif isinstance(block, SubsubheaderBlock):
        return f"### {block.title}"

    elif isinstance(block, QuoteBlock):
        return f"> {block.title}"

    elif isinstance(block, BulletedListBlock):
        return f"- {block.title}"

    elif isinstance(block, NumberedListBlock):
        return f"{counter}. {block.title}"

    elif isinstance(block, CodeBlock):
        return f"""
```{block.language}
{block.title}
```
"""

    elif isinstance(block, CalloutBlock):
        return f"> {block.icon} {block.title}"

    elif isinstance(block, DividerBlock):
        return "\n"
    else:
        return ""

In [62]:
#export
def page2md(page:PageBlock) -> str:
    """Translates a Notion Page (`PageBlock`) into a Markdown string."""
    blocks = list()

    # Numbered lists iterator
    counter = 1

    for block in page.children:
        blocks.append(block2md(block, counter))

        if isinstance(block, NumberedListBlock):
            counter += 1
        else:
            counter = 1

    return page_front_matter(page) + "\n".join(blocks)

In [63]:
#export
def page2path(page:PageBlock, dest_dir:Path=Path(".")) -> Path:
    """Build a .md file path in `dest_dir` based on a Notion page metadata."""
    return dest_dir/Path("-".join(page.title.lower().replace(".", "").split(" "))+ ".md")

In [64]:
#export
def page_front_matter(page: PageBlock) -> str:
    """Builds a page front matter in a yaml-like format."""
    internals = ['published', 'title']
    renderables = { k:v for k,v in page.get_all_properties().items() if k not in internals }

    return f"""
---
{yaml.dump(renderables)}
---\n
"""

## Converting multiple pages


In [20]:
#exports
def notion2md(token_v2:str, database_url:str, dest:Union[str, Path]) -> None:
    """
    Grab Notion Blog database using auth token `token_v2`,
    convert posts in database `database_url` to Markdown, and save them to `dest`.
    """

    client = notion_client(token_v2)

    database = notion_blog_database(client, database_url)

    for post in database.get_rows():
        page2path(page, dest_dir=dest).write_text(page2md(page))